In [1]:
import datascience 
import numpy as np
import graphviz

from datascience import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
  
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.base import ClassifierMixin
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
  
import pandas as pd
import numpy as np

from sklearn.externals import joblib

import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [2]:
t1=Table.read_table('W:\Climate1_.csv' , sep=",")

In [3]:
t2 = Table.read_table('W:\ClimateBal.csv' , sep=",")

In [4]:
t3 = t1.append(t2)

In [5]:
t3.to_csv("MergedLabelledData.csv")

In [6]:
t3_positive = t3.where('Support', are.equal_to(1))
t3_negative = t3.where('Support', are.equal_to(0))
t3_positive

ID,Text,Support
1075_Cleand_Climate1.csv,RT @sydneyleemarco: nothing like an 80 degree october da ...,1
85_Cleand_Climate1.csv,@MerlenesMemos @CNN It's not an act of god. Climate chan ...,1
916_Cleand_Climate1.csv,RT @gq_jayq: Bet I got 11 years to run it up https://t.c ...,1
557_Cleand_Climate1.csv,RT @HazelMonforton: Literally none of these things will ...,1
39_Cleand_Climate1.csv,@BjornLomborg I think looking at climate change in terms ...,1
608_Cleand_Climate1.csv,RT @KateRaworth: If you feel the dissonance between the ...,1
121_Cleand_Climate1.csv,@Weinsteinlaw His base doesn t care. unaffiliated who su ...,1
1070_Cleand_Climate1.csv,RT @smilon713: I wish NO harm to those in red states. I ...,1
80_Cleand_Climate1.csv,@LynchpinL With climate change making the world essentia ...,1
1121_Cleand_Climate1.csv,RT @yeson1631: .@VanJones68 all the life we know is deep ...,1


In [7]:
data = t3_positive.append(t3_negative)

In [8]:
data_tagged_X = list(data['Text'])
data_tagged_Y = list(data['Support'])
print('tagged data input size', len(data_tagged_X))
print('tagged data target prediction size', len(data_tagged_Y))

tagged data input size 2883
tagged data target prediction size 2883


In [9]:
clf=Pipeline([('vect',CountVectorizer(stop_words=[("a", "a"), ("an", "n"), ("the", "e"), ("this", "s"), ("that", "t"), ("is", "s")], token_pattern='(([#@]|[0-9]|[a-z]|[A-Z])+)',
                                            analyzer='word',min_df = 10 )),
                   ('clf',tree.DecisionTreeClassifier(criterion='entropy',random_state=100,max_depth=7,
                      min_samples_leaf=10))])
clf.fit(data_tagged_X,data_tagged_Y )

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=[('a', 'a'...       min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best'))])

In [10]:
skf = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)


In [11]:
fold = 1
precision = []
recall = []
f1 = []
for train_index, test_index in skf.split(data_tagged_X, data_tagged_Y):
    print("Fold Number:", fold)
    print("Training Data Index:", train_index)
    print("Testing Data Index:", test_index)
    
    x_train = list(data.take(train_index)['Text'])
    y_train = list(data.take(train_index)['Support'])
    x_test = list(data.take(test_index)['Text'])
    y_test = list(data.take(test_index)['Support'])
    
    count_vect = CountVectorizer(stop_words=[("a", "a"), ("an", "n"), ("the", "e"), ("this", "s"), ("that", "t"), ("is", "s")],
                                              token_pattern='(([#@]|[0-9]|[a-z]|[A-Z])+)',analyzer='word',min_df= 10)
    X_word_vect = count_vect.fit_transform(x_train)
    clf = tree.DecisionTreeClassifier(criterion='entropy', random_state = 100, max_depth=7, min_samples_leaf = 10)
    clf.fit(X_word_vect, y_train) 
    dot_data = tree.export_graphviz(clf, out_file = None, feature_names = count_vect.get_feature_names())
    graph = graphviz.Source(dot_data)
    file_name = 'StanceClassifier-Fold' + str(fold)
    graph.render(file_name)
    print("Test data: Saving Decision Tree Visualization in ",file_name)
   
    x_test_word_vect= count_vect.transform(x_test)
    predicted_y_test = clf.predict(x_test_word_vect) 
    #print(\"Testing Data Prediciton Output:\", predicted_y_test)
    print(metrics.confusion_matrix(y_test, predicted_y_test))
   
    precision.append(precision_score(y_test, predicted_y_test))
    recall.append(recall_score(y_test, predicted_y_test))
    f1.append(f1_score(y_test,predicted_y_test)) 
    print("Precision Score:", precision_score(y_test, predicted_y_test))
    print("Recall score:", recall_score(y_test, predicted_y_test))
    print("f1 Score:", f1_score(y_test, predicted_y_test))
    fold = fold+1

Fold Number: 1
Training Data Index: [   0    1    2 ... 2879 2881 2882]
Testing Data Index: [   3   12   19   37   47   48   49   51   56   60   65   73   75   80
   81   87   88   91   94   98  101  107  108  111  115  119  120  131
  133  135  142  159  163  167  169  177  181  186  190  194  195  198
  201  204  215  223  231  236  248  255  258  259  267  268  270  280
  283  288  292  298  301  302  304  309  311  321  325  330  335  341
  348  350  368  372  375  383  386  390  399  401  403  404  408  409
  418  419  422  439  442  443  453  460  464  471  480  484  487  490
  494  496  498  503  509  512  527  529  531  536  537  541  547  549
  556  561  571  572  573  575  577  581  589  613  628  631  634  636
  639  644  647  653  670  672  675  680  682  701  702  703  705  713
  726  730  735  736  737  757  764  765  766  778  779  780  782  785
  786  791  797  800  812  817  823  825  826  828  834  857  859  861
  862  864  875  881  888  892  895  898  909  920  926 

Test data: Saving Decision Tree Visualization in  StanceClassifier-Fold3
[[208  87]
 [136 146]]
Precision Score: 0.6266094420600858
Recall score: 0.5177304964539007
f1 Score: 0.566990291262136
Fold Number: 4
Training Data Index: [   0    1    2 ... 2880 2881 2882]
Testing Data Index: [   4    7    9   14   18   22   26   29   30   36   38   39   42   52
   67   71   82   93   95  102  105  112  116  122  124  128  147  153
  157  158  162  168  173  179  184  197  203  212  213  214  217  221
  225  227  232  249  261  271  272  273  284  285  287  296  300  305
  312  328  333  334  337  342  343  344  346  352  353  356  359  360
  361  362  365  371  373  377  378  379  380  392  394  396  397  402
  406  412  421  427  447  448  449  454  462  466  467  476  488  499
  500  501  504  506  510  511  514  517  518  519  522  523  533  540
  543  544  551  557  563  565  574  590  592  600  603  605  608  611
  618  635  645  658  662  667  676  677  684  688  691  704  716  721
  723

In [12]:
print("Precision List:", precision)
print("Recall List", recall)
print("f1 List:", f1)

Precision List: [0.5642023346303502, 0.6234309623430963, 0.6266094420600858, 0.624031007751938, 0.6810344827586207]
Recall List [0.5141843971631206, 0.5283687943262412, 0.5177304964539007, 0.5709219858156028, 0.5602836879432624]
f1 List: [0.5380333951762523, 0.5719769673704415, 0.566990291262136, 0.5962962962962962, 0.6147859922178989]


In [13]:
print("Average Precision:", np.mean(precision))
print("Average Recall:", np.mean(recall))
print("Average F1:", np.mean(f1))

Average Precision: 0.6238616459088182
Average Recall: 0.5382978723404255
Average F1: 0.577616588464605


In [14]:
data_entire_X = list(data['Text'])
data_entire_Y = list(data['Support'])

In [15]:
clf=Pipeline([('vect',CountVectorizer(stop_words=[("a", "a"), ("an", "n"), ("the", "e"), ("this", "s"), ("that", "t"), ("is", "s")], token_pattern='(([#@]|[0-9]|[a-z]|[A-Z])+)',
                                            analyzer='word',min_df = 10 )),
                   ('clf',tree.DecisionTreeClassifier(criterion='entropy',random_state=100,max_depth=7,
                      min_samples_leaf=10))])
clf.fit(data_entire_X,data_entire_Y )

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=[('a', 'a'...       min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best'))])

In [16]:
joblib.dump(clf, 'ClimateTeam20PD2A.pkl')

['ClimateTeam20PD2A.pkl']

In [17]:
clf2 = joblib.load('ClimateTeam20PD2A.pkl')

In [18]:
predicted_Y = clf2.predict(data_entire_X)

In [19]:
 print("f1 Score of labelled data:", f1_score(data_tagged_Y, predicted_Y))

f1 Score of labelled data: 0.6301067353698933
